# Figure 2

- Notebook to make data for figure 2 (likely a map)
- by Cascade Tuholske on 2020.10.01

#### Depdencies

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import statsmodels.api as sm
import seaborn as sns
import glob

#### Functions

In [2]:
def make_pdays(df_stats, df_pop):
    
    "Makes a dataframe with stats and population to calc people days"
    
    # Make Population Long Format
    pop_long = pd.wide_to_long(df_pop, stubnames = 'P', i = 'ID_HDC_G0', j = 'year')
    pop_long.reset_index(level=0, inplace=True)
    pop_long.reset_index(level=0, inplace=True)
    pop_long = pop_long.drop('Unnamed: 0', axis = 1)
    
    # Get Total Days 
    data = df_stats.drop_duplicates(['year', 'ID_HDC_G0'], keep = 'first') # drop out duplicates
    pdays = pd.DataFrame()
    pdays['ID_HDC_G0'] = data['ID_HDC_G0']
    pdays['year'] = data['year']
    pdays['total_days'] = data['total_days']
    pdays['CTR_MN_NM'] = data['CTR_MN_NM']
    pdays['sub-region'] = data['sub-region']
    pdays['region'] = data['region']
    pdays['intermediate-region'] = data['intermediate-region']
    
    # Merge
    pdays_merge = pdays.merge(pop_long, on=['ID_HDC_G0', 'year'], how = 'left')

    # Now get people days from 1983 and change
    p = pd.DataFrame()
    p['ID_HDC_G0'] = df_pop['ID_HDC_G0']
    p['P1983'] = df_pop['P1983']
    p['P2016'] = df_pop['P2016']
#    p['p_delt'] = (df_pop['P2016'] - df_pop['P1983']) / df_pop['P1983'] * 100

    pdays_merge = pdays_merge.merge(p ,on=['ID_HDC_G0'], how = 'left')
    
    # Calc p days
    pdays_merge['people_days'] = pdays_merge['total_days'] * pdays_merge['P'] #/ 10**9 # total people days
    pdays_merge['people_days_83'] = pdays_merge['total_days'] * pdays_merge['P1983'] #/ 10**9 # people days w/ pop con
    pdays_merge['people_days_attr'] = pdays_merge['people_days'] - pdays_merge['people_days_83']  # dif
    
    return pdays_merge

In [ ]:
# Regressions

# Get Data

def lm_func(df, col):
    
    "simple linear model of a time series data, returns coef"
    
    X_year = np.array(df.groupby('year')['ID_HDC_G0'].mean().index).reshape((-1, 1))
    Y_stats = np.array(df.groupby('year')[col].sum()).reshape((-1, 1))

    lm = LinearRegression()
    model = lm.fit(X_year, Y_stats)
    coef = lm.coef_
    
    
    return coef

# Load and Make Data

In [3]:
# file paths
DATA_IN = "/home/cascade/projects/data_out_urbanheat/"  # Note: Need ?dl=1 to make sure this file gets read correctly
FN_stats = 'heatrange/All_data20200109_406C_es_final.csv'
df_stats = pd.read_csv(DATA_IN+FN_stats)
# FIG_OUT = '/home/cascade/projects/figures/'

FN_pop = 'GHS-UCDB-Interp.csv'
df_pop = pd.read_csv(DATA_IN+FN_pop)

In [4]:
df_data = make_pdays(df_stats, df_pop)

In [5]:
df_data.head()

,ID_HDC_G0,year,total_days,CTR_MN_NM,sub-region,region,intermediate-region,P,P1983,P2016,people_days,people_days_83,people_days_attr
0,2784,1983,1,Germany,Western Europe,Europe,Western Europe,9.976361e+05,997636.118717,1.573652e+06,9.976361e+05,997636.118717,0.000000
1,2784,2013,1,Germany,Western Europe,Europe,Western Europe,1.538112e+06,997636.118717,1.573652e+06,1.538112e+06,997636.118717,540475.589177
2,2833,1983,1,Germany,Western Europe,Europe,Western Europe,6.014093e+04,60140.932532,6.497856e+04,6.014093e+04,60140.932532,0.000000
3,2833,1984,1,Germany,Western Europe,Europe,Western Europe,6.033127e+04,60140.932532,6.497856e+04,6.033127e+04,60140.932532,190.332532
4,2833,1998,1,Germany,Western Europe,Europe,Western Europe,6.411484e+04,60140.932532,6.497856e+04,6.411484e+04,60140.932532,3973.903796


# Test Regression Results

In [6]:
# SK LEARN

In [8]:
# Code source: Jaques Grobler
# License: BSD 3 clause


import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

# Load the diabetes dataset
diabetes_X, diabetes_y = datasets.load_diabetes(return_X_y=True)

# Use only one feature
diabetes_X = diabetes_X[:, np.newaxis, 2]

# Split the data into training/testing sets
diabetes_X_train = diabetes_X[:-20]
diabetes_X_test = diabetes_X[-20:]

# Split the targets into training/testing sets
diabetes_y_train = diabetes_y[:-20]
diabetes_y_test = diabetes_y[-20:]

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(diabetes_X_train, diabetes_y_train)

# Make predictions using the testing set
diabetes_y_pred = regr.predict(diabetes_X_test)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(diabetes_y_test, diabetes_y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(diabetes_y_test, diabetes_y_pred))

Coefficients: 
 [938.23786125]
Mean squared error: 2548.07
Coefficient of determination: 0.47


In [9]:
# Code source: Jaques Grobler
# License: BSD 3 clause


import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

# Load the diabetes dataset
diabetes_X, diabetes_y = datasets.load_diabetes(return_X_y=True)

# Use only one feature
diabetes_X = diabetes_X[:, np.newaxis, 2]

# Split the data into training/testing sets
diabetes_X_train = diabetes_X[:-20]
diabetes_X_test = diabetes_X[-20:]

# Split the targets into training/testing sets
diabetes_y_train = diabetes_y[:-20]
diabetes_y_test = diabetes_y[-20:]

# Create linear regression object
model = sm.OLS(diabetes_y_train, diabetes_X_train).fit()
coef = model.params[0]
r2 = model.rsquared_adj
p = model.pvalues[0]

# The coefficients
print('Coefficients: \n', coef)


Coefficients: 
 970.1672312903979


# Create Reg Stats

In [ ]:
## Slope of increase in days 

# make things easier for OLS
regressor = LinearRegression()  

# Geog 
geog = 'sub-region'

# Set stat
stats = 'people_days_83'

# Get results

labels = []
delt_list = []
r2_list = []
coef_list = []
p_list = []
df_out = pd.DataFrame()

for label, df in df_data.groupby(geog):
    
    # Get Data
    X_year = np.array(df.groupby('year')['ID_HDC_G0'].mean().index).reshape((-1, 1))
    Y_stats = np.array(df.groupby('year')[stats].sum()).reshape((-1, 1))
    
#     # Regress
    model = sm.OLS(Y_stats, X_year).fit()
    coef = model.params[0]
    r2 = model.rsquared_adj
    p = model.pvalues[0]

#     # Regress "simple linear model of a time series data, returns coef"
#     lm = LinearRegression()
#     model = lm.fit(X_year, Y_stats)
#     coef = lm.coef_[0][0]
    #r2 = lm.score
    
    # Pop change
    delt = (df['P2016'].sum() - df['P1983'].sum())/df['P1983'].sum() * 100
    
    # Make lists
    labels.append(label)
    r2_list.append(r2)
    coef_list.append(coef)
    #p_list.append(p)
    delt_list.append(delt)


# Make data frame
df_out[geog] = labels
df_out['p_delt'] = delt_list
#df_out['r2'] = r2_list
df_out['coef'] = coef_list
#df_out['p_value'] = [round(elem, 4) for elem in p_list]
df_out.head(14)

# What is the relationship between population exposed and increase in days?

In [ ]:
fig, ax1 = plt.subplots(1, figsize = (10,10), sharex=True)

ax1.scatter(df_out['p_delt'], df_out['coef'])
plt.xscale('log')
plt.yscale('log')
plt.xlim(1, 10000)

# Title
ax1.set_title('Change in Population vs Change in Extreme Heat', fontsize = 20)
ax1.set_ylabel('Additional Person Days [Billions] due to heat', fontsize = 20)
ax1.set_xlabel('Pct. Change in Population Exposed 1983 - 2016', fontsize = 20)
ax1.tick_params(labelsize = 15)